In [1]:
# To support both python 2 and python 3
# 让这份笔记同步支持 python 2 和 python 3
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
#使用statsmodels便于统计分析数据
import statsmodels.api as sm
import statsmodels.formula.api as smf
#使用sklearn便于预测（机器学习）
from sklearn import linear_model
#使用patsy便于生成模型
import patsy

In [3]:
np.random.seed(1)
columns_name = ['y']
for i in range(1,21):
    columns_name.append('X' + str(i))
X = pd.DataFrame(np.random.normal(0.0, 1.0, (1000,20)),columns = columns_name[1:])
epsilon = np.random.normal(0.0, 1.0, 1000)
belta = pd.DataFrame(np.array([[8,3,0,4,2,7,8,8,0,3,1,1,2,0,1,2,0,8,0,2]]),columns = columns_name[1:])
#y = (belta * X).sum(axis = 1) + epsilon
y = pd.DataFrame((belta.dot(X.T) + epsilon).T)

In [4]:
from sklearn.feature_selection import RFECV
estimator = linear_model.LinearRegression()
selector = RFECV(estimator, step=1, cv=10)
selector = selector.fit(X, y)
selector.support_

c:\users\wujiaxu\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True, False,  True,
       False,  True])

In [5]:
selector.ranking_

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 3, 1, 4, 1])

In [6]:
selector.estimator_.coef_

array([8.03640101, 2.9797092 , 3.95931347, 1.96472685, 7.01189163,
       7.94604801, 8.02340779, 0.08069804, 2.96447921, 0.99812975,
       0.93839843, 2.07440712, 1.01668718, 2.00506937, 7.95927483,
       2.04464643])

In [7]:
belta_hat = belta.copy()
j = 0
for i in range(20):
    column = belta.columns[i]
    if selector.ranking_[i] == 1:
        belta_hat[column] = selector.estimator_.coef_[j]
        j = j + 1
    else:
        belta_hat[column] = 0

sklearn实现子集选择-交叉验证的方式效率很高

In [8]:
selector.estimator_.intercept_

-0.05740809334269337

In [9]:
pd.concat([belta,belta_hat,belta_hat - belta],ignore_index = True ).rename(index = {0:r"$\beta$",1:r"$\beta_h$",2:'$\delta$'}).T

,$\beta$,$\beta_h$,$\delta$
X1,8.0,8.036401,0.036401
X2,3.0,2.979709,-0.020291
X3,0.0,0.000000,0.000000
X4,4.0,3.959313,-0.040687
X5,2.0,1.964727,-0.035273
X6,7.0,7.011892,0.011892
X7,8.0,7.946048,-0.053952
X8,8.0,8.023408,0.023408
X9,0.0,0.080698,0.080698
X10,3.0,2.964479,-0.035521


**实现速度较宽，而且也只有X9和截距多出了拟合**